In [1]:
print("The isochronal map here can be reproduced in the example 3DVectorVisualization")

The isochronal map here can be reproduced in the example 3DVectorVisualization


In [2]:


import sys
import os
sys.path.append(os.path.abspath("../"))
from electrophysiology_mapping.map import *
import numpy as np


In [3]:
print("Define Fitting Functions")

def setup_scene_and_capture(points, us, vs, scale=0.04,rotate_x = 0, 
                            rotate_y = 0, rotate_z = 200, radius=0.8, 
                            max_cv=None, min_cv=None, texture_file="contour.png",
                            save_dir = "vector_on_sphere"
                            ):
    """Sets up the renderer, renders the window with arrows, rotates, captures screenshots, and saves them to a folder."""
    # Create renderer and render window
    renderer = vtk.vtkRenderer()
    render_window = vtk.vtkRenderWindow()
    render_window.SetSize(1200, 1200)  # Larger window size
    render_window.AddRenderer(renderer)
    renderer.SetBackgroundAlpha(0)  
    renderer.SetBackground(1, 1, 1)  # Set the background to white
    render_window_interactor = vtk.vtkRenderWindowInteractor()
    render_window_interactor.SetRenderWindow(render_window)
    render_window.SetAlphaBitPlanes(1)  # Enable alpha channel
    render_window.SetMultiSamples(0)   # Disable multi-sampling for better transparency


    # Setup camera
    camera = renderer.GetActiveCamera()
    camera.ParallelProjectionOn()
    camera.SetPosition(0, 0, 10)  # Observe from bottom
    camera.SetFocalPoint(0, 0, 0)  # Look at the center of the sphere
    camera.SetViewUp(0, 1, 0)     # Set the upward direction
    renderer.AutomaticLightCreationOff()
    renderer.UseShadowsOff()
    renderer.SetAmbient(1.0, 1.0, 1.0)
    # Render objects
    create_sphere(renderer, radius=radius, resolution=100, rotate_x=rotate_x, rotate_y=rotate_y, rotate_z=rotate_z, texture_file=texture_file)
    for idx, (lon, lat) in enumerate(points):
        position = lat_lon_to_cartesian(lat, lon, radius)
        north = north_vector(lat, lon, radius)
        east = east_vector(position, north)
        direction = us[idx] * east + vs[idx] * north
        amplitude = np.sqrt(np.sum(us[idx]**2 + vs[idx]**2))
        normalized_intensity = (amplitude - min_cv) / (max_cv - min_cv) if max_cv and min_cv else (amplitude - 10) / (90)
        normalized_intensity = np.clip(normalized_intensity, 0, 1)
        color = jet_colormap(normalized_intensity)
        direction = direction / np.linalg.norm(direction)
        add_arrow(renderer, position, direction, scale * np.log(np.log(amplitude) + 5), color)

    # Create directory if it doesn't exist
    directory = save_dir
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Initialize Window to Image Filter
    w2if = vtk.vtkWindowToImageFilter()
    w2if.SetInput(render_window)
    w2if.SetScale(1)  # Image quality
    w2if.SetInputBufferTypeToRGBA()
    w2if.ReadFrontBufferOff()
    
    # Initialize the writer
    writer = vtk.vtkPNGWriter()

    # Rotate and capture images
    for i in range(36):  # 360 degrees in steps of 10 degrees
        camera.Azimuth(10)  # Rotate camera
        render_window.Render()  # Render the scene
        w2if.Modified()  # Update the filter to reflect new scene
        w2if.Update()  # Ensure the filter processes the latest image
        writer.SetInputConnection(w2if.GetOutputPort())
        writer.SetFileName(f"{directory}/screenshot_{i}.png")
        writer.Write()

    render_window_interactor.Start()




Define Fitting Functions


In [4]:
setup_scene_and_capture([],[],[],
                  scale = 0.04, rotate_x= 0, rotate_y= 0, rotate_z = 200
                  , radius= 0.8, min_cv=1, max_cv=10,texture_file="isochronalMap.png",
                  save_dir= "2DMapping")

In [5]:
setup_scene_and_capture([],[],[],
                  scale = 0.04, rotate_x= 0, rotate_y= 0, rotate_z = 200
                  , radius= 0.8, min_cv=1, max_cv=10,texture_file="random_map_with_electrode.png",
                  save_dir="2DMappingWithElectrodePosition")
